# Einführung in OpenCV 

In [ ]:
import sys

%conda install --yes --prefix {sys.prefix} -c conda-forge opencv

In [ ]:
import imageio
import cv2
import numpy as np
from matplotlib import pyplot as plt

Beispielbilder werden mit imageio frei verschifft.
Für die Übersicht der Bilder siehe [hier](https://imageio.readthedocs.io/en/latest/standardimages.html).
Zunächst wird das Bild mithilfe von matplotlib visualisiert, das eine gute Integration mit Jupyter Notebooks hat.
[Hier](https://medium.com/@mrdatainsight/how-to-use-opencv-imshow-in-a-jupyter-notebook-quick-tip-ce83fa32b5ad) gibt es weiterführende Informationen, wobei es bei OpenCV hakelt.

In [ ]:
gray_image = imageio.imread("imageio:coins.png")
plt.imshow(gray_image, cmap="gray")
plt.show()

Nun ist bereits thematisiert worden, dass Bilder Matrizen mit Zahlenwerten sind.
Es gibt verschiedene Darstellungsformen, weswegen wir nun schauen, welche hier vorliegt.

In [ ]:
gray_image.min(), gray_image.max()

Die Verteilung der Intensitätswerte kann man sich über ein Histogramm verdeutlichen.

In [ ]:
plt.hist(gray_image.flatten())
plt.ylabel("Häufigkeit")
plt.xlabel("Intensitätswert der Pixel")
plt.show()

$\rightarrow$ Es ist die Darstellung zwischen 0 und 255 - die Zahlen repräsentieren die Intensitätswerte.

## Canny-Filter

Aber wie kann man nun die Münzen automatisiert detektieren?
Der Fragezeichenoperator ruft die Dokumentation einer Klasse oder einer Funktion auf.

In [ ]:
?cv2.Canny

In [ ]:
edged_1 = cv2.Canny(gray_image, 1, 1)
plt.title('Canny edges (1, 1)')
plt.imshow(edged_1, cmap="gray")
plt.show()

edged_2 = cv2.Canny(gray_image, 255, 255)
plt.title('Canny edges (255, 255)')
plt.imshow(edged_2, cmap="gray")
plt.show()

**Aufgabe 1)**

Auftrag:
Finden Sie nun Werte für den Canny-Filter, mit dem man die Münzen am besten vom Hintergrund automatisch ausschneiden könnte.
Die Verwendung einer `for`-Schleife kann dabei helfen, schneller mehr Filter auszuprobieren.

In [ ]:
# Probieren Sie sich hier aus. Verwenden Sie bei Bedarf auch gerne mehrere Zellen.

## Blob-Detektor

Um Objekte zu identifizieren, stellt opencv Werkzeuge wie den Blob-Detektor bereit.
Dieser ermittelt, wo gleichartige Flächen sind.
Dies ist ein Algorithmus der traditionellen Bildverarbeitung und verwendet *kein* Maschinelles Lernen.

In [ ]:
detector = cv2.SimpleBlobDetector_create()
keypoints = detector.detect(gray_image)

[k.pt for k in keypoints]

Dies kann man nun auch in das Bild einzeichnen

In [ ]:
blobs = cv2.drawKeypoints(
    gray_image,
    keypoints,
    np.zeros((1, 1)),
    (255, 0, 0),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
plt.imshow(blobs)
plt.show()

Das Ergebnis ist noch nicht ganz überzeugend.
Die zwei Punkte sind eingezeichnet worden - 
der eine Blob ist eine Münze, der zweite aber nur ein kleiner Ausschnitt auf der zweiten Münze von rechts oben.

Die Standard-Parameter, die verwendet werden, wenn keine Werte angegeben werden, sind vielleicht nicht so gut geeignet.
Nun passen wir manuell einzelne Parameter an.
Weitere Informationen zur API von OpenCV gibt es [online](https://docs.opencv.org/3.4/d0/d7a/classcv_1_1SimpleBlobDetector.html), Beispiele bei stackoverflow.

In [ ]:
params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.filterByCircularity = False
params.filterByColor = True
params.filterByConvexity = False
params.filterByInertia = False

params.minArea = 250
params.minThreshold = 15
params.minDistBetweenBlobs = 10

detector = cv2.SimpleBlobDetector_create(params)

keypoints = detector.detect(gray_image)
[(i + 1, k.pt) for i, k in enumerate(keypoints)]

Auch dies können aufgrund der zu geringen Anzahl nicht die 24 Münzen sein.

In [ ]:
blobs = cv2.drawKeypoints(
    gray_image,
    keypoints,
    np.zeros((1, 1)),
    (255, 0, 0),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
plt.imshow(blobs)
plt.show()

Auch hier gibt es noch viele Baustellen:
Manche Münzen werden gar nicht erkannt, manchmal werden nur Teile der Münze eingekreist u. ä.
Dabei gibt es für den Blob-Detektor [gute Beispiele](https://www.learnopencv.com/blob-detection-using-opencv-python-c/) dafür, dass die Methode an sich funktioniert.

## Verknüpfe Canny-Filter und Blob-Detektor

Nun kann man die Stärken verschiedener Methoden miteinander kombinieren.
So kann man auch den Filter und den Detektor nacheinander ausführen.
Das Bild wird zuerst gefiltert und der Detektor wird mit dem gefilterten Bild aufgerufen.

In [ ]:
params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.filterByCircularity = True
params.filterByColor = False
params.filterByConvexity = False
params.filterByInertia = False

params.minArea = 150
params.minThreshold = 5
params.minDistBetweenBlobs = 5
params.minCircularity = 0.3

detector = cv2.SimpleBlobDetector_create(params)

keypoints = detector.detect(edged_2)
[(i + 1, k.pt) for i, k in enumerate(keypoints)]

In [ ]:
blobs = cv2.drawKeypoints(
    gray_image,
    keypoints,
    np.zeros((1, 1)),
    (255, 0, 0),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
plt.imshow(blobs)
plt.show()

Nun gibt es verschiedene Möglichkeiten, wie man weiterarbeiten kann.
Die Parameter können nun weiter "getunt" werden, d. h. weiter anpassen, bis das gewünschte Ergebnis erreicht wird.
Allerdings können auch der Filter und der Detektor ausgetauscht werden.
OpenCV bietet viel mehr Möglichkeiten, und es gibt auch weitere Bibliotheken.
Mit der Bibliothek `numpy` kann man sogar selbst Filter und Detektoren implementieren.

**Aufgabe 2a)**

Frage:
Wie lautet Ihre Hypothese, weswegen das Erkennen der Münzen auf dem Bild so schwierig ist?

Antwort: ...

**Aufgabe 2b)**

Frage:
Wie würden Sie in Zukunft den Prozess der Bildaufnahme gestalten, um die Erkennung der Münze zu vereinfachen?

Antwort: ...

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.